In [ ]:
### This script will plot some graphics showing the relative locations of the ZDR and KDP maxes in the storm.
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from metpy.units import atleast_1d, check_units, concatenate, units
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from siphon.radarserver import RadarServer
rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
#rs = RadarServer('http://thredds.ucar.edu/thredds/radarServer/nexrad/level2/IDD/')
from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from metpy.units import atleast_1d, check_units, concatenate, units
from shapely.geometry import polygon as sp
import pyproj 
import shapely.ops as ops
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from functools import partial
from shapely import geometry
import netCDF4

In [ ]:
query = rs.query()
#Here, set the initial time of the archived radar loop you want.
dt = datetime(2017, 7, 11, 23, 0) # Our specified time
station = 'KMVX'
query.stations(station).time_range(dt, dt + timedelta(hours=3.5))
cat = rs.get_catalog(query)
cat.datasets
f = 27
n = 1
for item in sorted(cat.datasets.items()):
    # After looping over the list of sorted datasets, pull the actual Dataset object out
    # of our list of items and access over CDMRemote
    try:
        ds = item[1]
        radar1 = pyart.io.nexrad_cdm.read_nexrad_cdm(ds.access_urls['OPENDAP'])
        #Now let's calculate and plot specific differential phase.
        for i in range(radar1.nsweeps):
            print('in loop')
            print(radar1.nsweeps)
            radar = radar1.extract_sweeps([i])
            if ((np.mean(radar.elevation['data']) < .52) and (np.max(np.asarray(radar.fields['differential_reflectivity']['data'])) != np.min(np.asarray(radar.fields['differential_reflectivity']['data'])))):
                print(np.mean(radar.elevation['data']))
                time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
                print(time_start)
                kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar)
                print('its this line')
                radar.add_field('KDP', kdp_dict[0])
                print('heres the problem')
                # mask out last 10 gates of each ray, this removes the "ring" around the radar.
                radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked

                # exclude masked gates from the gridding
                gatefilter = pyart.filters.GateFilter(radar)
                gatefilter.exclude_masked('differential_reflectivity')
                print('almost gridding')
                #Now let's grid the data
                grid = pyart.map.grid_from_radars(
                    (radar,), gatefilters=(gatefilter, ),
                    grid_shape=(1, 500, 500),
                    grid_limits=((200, 200), (-123000.0, 123000.0), (-123000.0, 123000.0)),
                    fields=['differential_reflectivity','reflectivity','KDP'])

                #Get the data from the grid
                ZDR = grid.fields['differential_reflectivity']['data'][0]
                REF = grid.fields['reflectivity']['data'][0]
                KDP = grid.fields['KDP']['data'][0]

            #Mask everything below 20dbz
            #import numpy.ma as ma
                ZDRmasked = ma.masked_where(REF < 20, ZDR)
                REFmasked = ma.masked_where(REF < 20, REF)
                KDPmasked = ma.masked_where(REF < 20, KDP)

                rlons = grid.point_longitude['data']
                rlats = grid.point_latitude['data']
                cenlat = radar.latitude['data'][0]
                cenlon = radar.longitude['data'][0]
                #Let's set up the map projection!
                print('Set up our projection')
                crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=45.0)

                # Set up our array of latitude and longitude values and transform to 
                # the desired projection.

                tlatlons = crs.transform_points(ccrs.LambertConformal(central_longitude=265, central_latitude=25, standard_parallels=(25.,25.)),rlons[0,:,:],rlats[0,:,:])
                tlons = tlatlons[:,:,0]
                tlats = tlatlons[:,:,1]

                # Limit the extent of the map area, must convert to proper coords.
                LL = (cenlon-1.5,cenlat-1.5,ccrs.PlateCarree())
                UR = (cenlon+1.5,cenlat+1.5,ccrs.PlateCarree())
                print(LL)

                # Get data to plot state and province boundaries
                states_provinces = cfeature.NaturalEarthFeature(
                        category='cultural',
                        name='admin_1_states_provinces_lakes',
                        scale='50m',
                        facecolor='none')
                fname = 'cb_2016_us_county_20m/cb_2016_us_county_20m.shp'
                fname2 = 'cb_2016_us_state_20m/cb_2016_us_state_20m.shp'
                counties = ShapelyFeature(Reader(fname).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                states = ShapelyFeature(Reader(fname2).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                fig=plt.figure(n,figsize=(30.,25.))
                ax = plt.subplot(111,projection=ccrs.PlateCarree())
                ax.coastlines('50m',edgecolor='black',linewidth=0.75)
                #ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
                ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
                ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
                ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
                REFlevels = np.arange(20,73,2)
                print('plotting')
                ax.contourf(rlons[0,:,:],rlats[0,:,:],REFmasked,REFlevels,cmap = plt.cm.gist_ncar)
                ZDRlevels = np.arange(3, np.max(ZDRmasked)+((np.max(ZDRmasked))-3)/2, (np.max(ZDRmasked))-3)
                zdrlev = [3.0]
                kdplev = [1.5]
                #ZDRlevels = np.arange(3,5.5,.5)
                #ZDRlevels1 = np.arange(5,10,.5)
                KDPlevels = np.arange(.75, np.max(KDPmasked)+((np.max(KDPmasked))-1.5)/2, (np.max(KDPmasked))-1.5)
                #KDPlevels = np.arange(.75,1.75,.25)
                #KDPlevels1 = np.arange(1.5,10,.25)
                zdrc = ax.contour(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdrlev,linewidths = 5, colors='blue', alpha = .8)
                #zrdc = ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels,linewide = .01, colors='pink', alpha = .8)
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels1,linewide = .01, colors='crimson', alpha = .8)

                #kdpc = ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels,linewide = .01, colors ='green', alpha = .5)
                kdpc = ax.contour(rlons[0,:,:],rlats[0,:,:],KDPmasked,kdplev,linewidths = 5, colors='green', alpha = .8)
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                print('made it here')
                plt.savefig('testfig.png')

                proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                           pyproj.Proj(init='epsg:3857'))
                zdr_areas = []
                zdr_centroid_lon = []
                zdr_centroid_lat = []
                zdr_mean = []
                zdr_max = []
                print("here too")
                for col in zdrc.collections:
                    # Loop through all polygons that have the same intensity level
                    print('hi')
                    for contour_path in col.get_paths(): 
                        # Create the polygon for this intensity level
                        # The first polygon in the path is the main one, the following ones are "holes"
                        print('hi')
                        for ncp,cp in enumerate(contour_path.to_polygons()):
                            print('hi')
                            cpa = np.asarray(cp[:])
                            x = cpa[:,0]
                            y = cpa[:,1]
                            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                            if ncp == 0:
                                poly = new_shape
                                #print('hi')
                            else:
                                # Remove the holes if there are any
                                poly = poly.difference(new_shape)
                                #print('hi')

                        # do something with polygon
                        #print(poly.area) 
                        #print(poly.centroid.x)
                        s_new = transform(proj, poly)
                        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                        boundary = np.asarray(poly.boundary.xy)
                        polypath = Path(boundary.transpose())
                        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                        mean = np.mean(ZDRmasked[mask])
                        if projected_area > 20 * units('km^2') and mean > 3.0:
                            zdr_areas.append((projected_area))
                            zdr_centroid_lon.append((poly.centroid.x))
                            zdr_centroid_lat.append((poly.centroid.y))
                            zdr_mean.append((mean))
                            zdr_max.append((np.max(ZDRmasked[mask])))
                        #print(s_new)

                kdp_areas = []
                kdp_centroid_lon = []
                kdp_centroid_lat = []

                for col in kdpc.collections:
                    # Loop through all polygons that have the same intensity level
                    for contour_path in col.get_paths(): 
                        # Create the polygon for this intensity level
                        # The first polygon in the path is the main one, the following ones are "holes"
                        for ncp,cp in enumerate(contour_path.to_polygons()):
                            print(1)
                            cpa = np.asarray(cp[:])
                            x = cpa[:,0]
                            y = cpa[:,1]
                            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                            if ncp == 0:
                                poly = new_shape
                            else:
                                # Remove the holes if there are any
                                poly = poly.difference(new_shape)

                        # do something with polygon
                        #print(poly.area) 
                        print(poly.centroid.x)
                        s_new = transform(proj, poly)
                        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                        if projected_area > 20 * units('km^2'):
                            kdp_areas.append((projected_area))
                            kdp_centroid_lon.append((poly.centroid.x))
                            kdp_centroid_lat.append((poly.centroid.y))

                    #print(s_new)

                if np.asarray(zdr_centroid_lon).shape[0] > 0:
                    ax.scatter(zdr_centroid_lon, zdr_centroid_lat, marker = '*', s = 500, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                if np.asarray(kdp_centroid_lon).shape[0] > 0:
                    ax.scatter(kdp_centroid_lon, kdp_centroid_lat, marker = '^', s = 500, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                for i in enumerate(zdr_areas):
                    plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f / %.2f / %.2f" %(zdr_areas[i[0]].magnitude, zdr_mean[i[0]], zdr_max[i[0]]), size = 23)
                    #plt.annotate(zdr_areas[i[0]], (zdr_centroid_lon[i[0]],zdr_centroid_lat[i[0]]))
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                #plt.tight_layout()
                #plt.savefig('ZDRarcannotated.png')
                hour = time_start.hour
                if hour < 10:
                    hour = '0'+str(hour)
                minute = time_start.minute
                if minute < 10:
                    minute = '0'+str(minute)
                day = time_start.day
                if day < 10:
                    day = '0'+str(day)
                title_plot = plt.title('Radar Reflectivity, ZDR, and KDP '+str(time_start.year)+'-'+str(time_start.month)+'-'+str(time_start.day)+
                                           ' '+str(hour)+':'+str(minute)+' UTC', size = 25)
                plt.savefig('ZDRArc_comp'+station+str(time_start.year)+str(time_start.month)+str(day)+str(hour)+str(minute)+'.png')
                n = n+1
    except:
        continue

plt.show()

In [ ]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
zdr_areas = []
zdr_centroid_lon = []
zdr_centroid_lat = []
zdr_mean = []
zdr_max = []

for col in zdrc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
                print(new_shape)
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        boundary = np.asarray(poly.boundary.xy)
        polypath = Path(boundary.transpose())
        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
        mean = np.mean(ZDRmasked[mask])
        if projected_area > 20 * units('km^2') and mean > 3.0:
            zdr_areas.append((projected_area))
            zdr_centroid_lon.append((poly.centroid.x))
            zdr_centroid_lat.append((poly.centroid.y))
            zdr_mean.append((mean))
            zdr_max.append((np.max(ZDRmasked[mask])))
        #print(s_new)
print(zdr_areas)
print(zdr_centroid_lon)
print(zdr_centroid_lat)
print(zdr_mean)
print(zdr_max)

In [ ]:
for i in range(len(zdrc.collections)):
    p = zdrc.collections[i].get_paths()[0]
    v = p.vertices
    x = v[:,0]
    y = v[:,1]
    if len(x)>2:
        poly = sp.Polygon([(i[0], i[1]) for i in zip(x,y)])
        print(i, poly.area)

In [ ]:
print(len(zdrc.collections))

In [ ]:
print(enumerate(contour_path.to_polygons()))
for cp in enumerate(contour_path.to_polygons()):
    print(1)

In [ ]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
kdp_areas = []
kdp_centroid_lon = []
kdp_centroid_lat = []

from shapely import geometry
for col in kdpc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print(1)
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        if projected_area > 20 * units('km^2'):
            kdp_areas.append((projected_area))
            kdp_centroid_lon.append((poly.centroid.x))
            kdp_centroid_lat.append((poly.centroid.y))

        #print(s_new)
print(kdp_areas)
print(kdp_centroid_lon)
print(kdp_centroid_lat)

In [ ]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
zdr_areas = []
zdr_centroid_lon = []
zdr_centroid_lat = []
zdr_mean = []
zdr_max = []

for col in zdrc.collections:
    # Loop through all polygons that have the same intensity level
    print('hi')
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        print('hi')
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print('hi')
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
                print('hi')
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)
                print('hi')

        # do something with polygon
        #print(poly.area) 
        #print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        boundary = np.asarray(poly.boundary.xy)
        polypath = Path(boundary.transpose())
        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
        mean = np.mean(ZDRmasked[mask])
        if projected_area > 20 * units('km^2') and mean > 3.0:
            zdr_areas.append((projected_area))
            zdr_centroid_lon.append((poly.centroid.x))
            zdr_centroid_lat.append((poly.centroid.y))
            zdr_mean.append((mean))
            zdr_max.append((np.max(ZDRmasked[mask])))
        #print(s_new)
print(zdr_areas)
print(zdr_centroid_lon)
print(zdr_centroid_lat)
print(zdr_mean)
print(zdr_max)

In [ ]:
query = rs.query()
#Here, set the initial time of the archived radar loop you want.
dt = datetime(2017, 7, 11, 23, 30) # Our specified time
station = 'KMVX'
query.stations(station).time_range(dt, dt + timedelta(hours=.1))
cat = rs.get_catalog(query)
cat.datasets
f = 27
n = 1
for item in sorted(cat.datasets.items()):
    # After looping over the list of sorted datasets, pull the actual Dataset object out
    # of our list of items and access over CDMRemote
    try:
        ds = item[1]
        radar = pyart.io.nexrad_cdm.read_nexrad_cdm(ds.access_urls['OPENDAP'])
    except:
        continue

In [ ]:
radar1 = radar.extract_sweeps([1])
print(np.mean(radar1.elevation['data']))

In [ ]:
print(radar.nsweeps)
print(np.max(np.asarray(radar1.fields['differential_reflectivity']['data'])))

In [ ]:
for i in range(radar.nsweeps):
        radar2 = radar.extract_sweeps([i])
        print(np.mean(radar2.elevation['data']))
        if ((np.mean(radar2.elevation['data']) < .52) and (np.max(np.asarray(radar2.fields['differential_reflectivity']['data'])) != 1.0)):
                print(np.mean(radar2.elevation['data']))
                time_start = netCDF4.num2date(radar2.time['data'][0], radar2.time['units'])
                print(time_start)
                kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar2)
                print('kdp worked')
        